In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote  # 이 부분 추가
import pandas as pd
import time
import logging
from datetime import datetime
import os
import threading
import re  # 코드 상단에 추가

# PDF 저장 디렉토리와 로그 디렉토리 설정
PDF_DIR = "data/skhynix"
LOG_DIR = "data/logs"

# 디렉토리 생성
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# 로그 설정
log_file = os.path.join(LOG_DIR, f"news_download_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        # logging.StreamHandler()
    ]
)

In [2]:
# WebDriver 인스턴스를 관리하는 싱글톤 클래스
class ChromeDriverWrapper:
    """Chrome WebDriver를 관리하는 싱글톤 클래스"""
    _instance = None
    _driver = None
    _lock = threading.Lock()
    
    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls()
        return cls._instance
    
    def get_driver(self):
        if self._driver is None:
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            
            self._driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()),
                options=chrome_options
            )
        return self._driver
    
    def quit_driver(self):
        if self._driver:
            try:
                self._driver.quit()
            except Exception as e:
                logging.error(f"드라이버 종료 중 오류: {str(e)}")
            finally:
                self._driver = None

In [3]:
class SKHynixNewsScraper:
    def __init__(self):
        logging.info("스크래퍼 초기화 시작")
        self.base_url = "https://news.skhynix.co.kr"
        
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        self.wait = WebDriverWait(self.driver, 20)
        self.articles = []
        logging.info("스크래퍼 초기화 완료")

    def get_article_content(self, url):
        """개별 기사의 내용을 가져오는 메서드"""
        logging.info(f"기사 내용 추출 시작: {url}")
        
        try:
            # 현재 창의 핸들 저장
            main_window = self.driver.current_window_handle
            
            # 새 창에서 링크 열기
            self.driver.execute_script(f"window.open('{url}', '_blank');")

            # 새 창으로 전환 전 지연
            time.sleep(3)            

            # 새 창으로 전환
            WebDriverWait(self.driver, 10).until(lambda driver: len(driver.window_handles) > 1)
            new_window = [handle for handle in self.driver.window_handles if handle != main_window][0]
            self.driver.switch_to.window(new_window)
            logging.info("새 창으로 전환 완료")

            # 페이지 로딩 대기
            time.sleep(5)            

            try:
                # 본문 컨테이너 대기 및 찾기
                content_container = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.post-contents"))
                )
                
                # 본문 내용 추출
                paragraphs = content_container.find_elements(By.TAG_NAME, "p")
                content_texts = []
                
                for p in paragraphs:
                    text = p.text.strip()
                    if text and not text.startswith('* '):  # 각주 제외
                        content_texts.append(text)
                
                content = '\n'.join(content_texts)
                
                if content:
                    logging.info(f"본문 내용 추출 성공 (길이: {len(content)} 자)")
                    return content
                else:
                    logging.error("본문 내용이 비어있습니다")
                    return "내용 추출 실패"
                
            except Exception as e:
                logging.error(f"본문 추출 중 오류: {str(e)}")
                # 디버깅을 위한 현재 페이지 HTML 로깅
                logging.debug(f"현재 페이지 HTML: {self.driver.page_source}")
                return "내용 추출 실패"
                
            finally:
                # 새 창 닫기
                self.driver.close()
                # 원래 창으로 돌아가기
                self.driver.switch_to.window(main_window)
                logging.info("원래 창으로 복귀 완료")
            
        except Exception as e:
            logging.error(f"기사 내용 추출 중 오류: {str(e)}")
            try:
                self.driver.switch_to.window(main_window)
                logging.info("에러 후 원래 창으로 복귀")
            except:
                logging.error("원래 창으로 복귀 실패")
            return "내용 추출 실패"

    def clean_content(self, content):
        """본문 내용 정제"""
        # 불필요한 공백 제거
        content = ' '.join(content.split())
        
        # 각주 표시(*) 제거
        content = re.sub(r'\*\s*[^*]+\*', '', content)
        
        # 빈 줄 제거
        content = re.sub(r'\n\s*\n', '\n', content)
        
        return content.strip()

    def search_news(self, keyword, page_limit=130):
        logging.info(f"뉴스 검색 시작 - 키워드: {keyword}, 페이지 제한: {page_limit}")
        current_page = 1
        encoded_keyword = quote(keyword)
        
        try:
            while current_page <= page_limit:

                # newsroom
                # if current_page == 1:
                #     url = f"{self.base_url}/?s={encoded_keyword}&type=newsroom"
                # ?s=*&start_date=2016.1&end_date=2024.12
                # else:
                #     url = f"{self.base_url}/page/{current_page}/?s={encoded_keyword}&type=newsroom"
                # s&start_date=2016.1&end_date=2024.12&type=newsroom
                # press
                if current_page == 1:
                    url = f"{self.base_url}/all/"
                else:
                    url = f"{self.base_url}/all/page/{current_page}/"
                
                logging.info(f"페이지 접근: {url}")
                self.driver.get(url)

                # 페이지 로딩 대기
                time.sleep(7)
                
                try:
                    articles = self.driver.find_elements(By.TAG_NAME, "article")
                    logging.info(f"현재 페이지에서 {len(articles)}개의 기사 발견")
                    
                    if not articles:
                        logging.info("더 이상 기사가 없습니다")
                        break
                    
                    for article in articles:
                        try:
                            # 기사 처리 전 지연
                            time.sleep(2)

                            # 제목과 URL 추출
                            title_selectors = [
                                (By.CSS_SELECTOR, "h2.tit a"),
                                (By.CSS_SELECTOR, "a.tit"),
                                (By.TAG_NAME, "a")
                            ]
                            
                            title = None
                            url = None
                            
                            for selector_type, selector in title_selectors:
                                try:
                                    link_element = article.find_element(selector_type, selector)
                                    title = link_element.text.strip()
                                    url = link_element.get_attribute("href")
                                    if title and url:
                                        break
                                except:
                                    continue
                            
                            if not title or not url:
                                continue
                            
                            # 날짜 추출
                            try:
                                date_element = article.find_element(By.CLASS_NAME, "date")
                                date = date_element.text.strip()
                            except:
                                date = "날짜 정보 없음"
                            
                            # 카테고리 추출
                            try:
                                category_elements = article.find_elements(By.CSS_SELECTOR, "div.category a")
                                categories = [cat.text.strip() for cat in category_elements]
                                category = " ".join(categories)
                            except:
                                category = "카테고리 없음"
                            
                            # 태그 추출
                            try:
                                tag_elements = article.find_elements(By.CSS_SELECTOR, "ul.tags li a")
                                tags = [tag.text.strip() for tag in tag_elements]
                                tags_text = ", ".join(tags)
                            except:
                                tags_text = "태그 없음"
                            
                            # 기사 내용 가져오기
                            content = self.get_article_content(url)
                            
                            article_data = {
                                "title": title,
                                "url": url,
                                "date": date,
                                "category": category,
                                "tags": tags_text,
                                "content": content
                            }
                            
                            self.articles.append(article_data)
                            logging.info(f"기사가 성공적으로 추가됨: {title}")
                            
                        except Exception as e:
                            logging.error(f"기사 정보 추출 중 오류: {str(e)}")
                    
                    # 다음 페이지 확인
                    # try:
                    #     next_page = self.driver.find_element(
                    #         By.CSS_SELECTOR,
                    #         f"div.list-pagination a.page[href*='/page/{current_page + 1}/']"
                    #     )
                    #     current_page += 1
                    #     logging.info(f"다음 페이지로 이동: {current_page}")
                    # except:
                    #     logging.info("마지막 페이지에 도달했습니다")
                    #     break
                    # 다음 페이지 확인
                    try:
                        if current_page % 10 == 0:
                            # 10의 배수 페이지에서는 next 버튼 클릭
                            next_button = self.driver.find_element(
                                By.CSS_SELECTOR,
                                "a.btn-nav.next"
                            )
                            next_button.click()
                            current_page += 1
                            logging.info(f"다음 10페이지로 이동: {current_page}")
                            time.sleep(3)  # 페이지 로딩 대기
                        else:
                            # 일반적인 다음 페이지 이동
                            next_page = self.driver.find_element(
                                By.CSS_SELECTOR,
                                f"div.list-pagination a.page[href*='/page/{current_page + 1}/']"
                            )
                            next_page.click()
                            current_page += 1
                            logging.info(f"다음 페이지로 이동: {current_page}")
                            time.sleep(3)  # 페이지 로딩 대기
                    except Exception as e:
                        logging.error(f"페이지 이동 중 오류: {str(e)}")
                        logging.info("마지막 페이지에 도달했습니다")
                        break
                                        
                except Exception as e:
                    logging.error(f"페이지 처리 중 오류: {str(e)}")
                    break
                
        except Exception as e:
            logging.error(f"검색 중 오류 발생: {str(e)}")
        
        finally:
            logging.info(f"검색 완료 - 총 {len(self.articles)}개의 기사 수집")

    def save_to_csv(self, filename):
        logging.info(f"CSV 파일 저장 시작: {filename}")
        try:
            if not self.articles:
                logging.warning("저장할 기사가 없습니다")
                return
                
            df = pd.DataFrame(self.articles)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            logging.info("CSV 파일 저장 완료")
        except Exception as e:
            logging.error(f"CSV 파일 저장 중 오류: {str(e)}")

    def __del__(self):
        try:
            self.driver.quit()
        except:
            pass

In [4]:
try:
    scraper = SKHynixNewsScraper()
    scraper.search_news("*", page_limit=130)
    scraper.save_to_csv("data/skhynix/leadership_news.csv")
finally:
    scraper.__del__()